In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://mongo1:27017/classwallet-spark.orders?replicaSet=rs0").\
        config("spark.mongodb.output.uri","mongodb://mongo1:27017/classwallet-spark.categories?replicaSet=rs0").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

In [3]:
pipeline = [{
    '$match': {'status': 'complete'}}, 
    { '$unwind':'$vendor.items'},
     {'$project': {
            'itemId':'$vendor.items.itemId',
            'total':1, 
            'customerId':1, 
            'completed': 1, 
            'vendorId': '$vendor.vendorId'
        }},
]


orders_df = spark.read.format("mongo").option("pipeline", pipeline).load()

In [4]:
orders_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- completed: timestamp (nullable = true)
 |-- itemId: string (nullable = true)
 |-- total: double (nullable = true)
 |-- vendorId: string (nullable = true)



In [21]:
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType

# def array_to_string(my_list):
#     return '[' + ','.join([str(elem) for elem in my_list]) + ']'

# array_to_string_udf = udf(array_to_string, StringType())

# orders_df = orders_df.withColumn('items_string', array_to_string_udf(orders_df['vendor.items']))

# orders_df.coalesce(1).write.csv('orders')

In [17]:
orders_df.count()


357648

In [14]:
!ls orders1a

_SUCCESS
